<a href="https://colab.research.google.com/github/anshuls08997/LibraryManagement/blob/main/StockPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Stock Price Prediction
!pip install -q yfinance plotly tensorflow scikit-learn

import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import plotly.graph_objects as go
from datetime import datetime, timedelta
import os
from google.colab import files
from tqdm import tqdm

# Configuration
config = {
    'ticker': 'AAPL',
    'start_date': '2020-01-01',
    'end_date': datetime.now().strftime('%Y-%m-%d'),
    'test_size': 0.2,
    'lookback': 60,
    'epochs': 10,
    'batch_size': 30,
    'prediction_days': 30,
    'features': ['Close', 'Volume'],
    'save_dir': '/content/sample_data/stock/stock_data.csv'  # New clean directory
}


try:
    os.makedirs(config['save_dir'], exist_ok=True)
    print(f"✅ Successfully created directory: {config['save_dir']}")
except Exception as e:
    print(f"⚠️ Couldn't create directory: {e}")
    # Fallback to Colab's default directory
    config['save_dir'] = '/content'
    print(f"Using fallback directory: {config['save_dir']}")

# Set file paths
csv_path = os.path.join(config['save_dir'], f"{config['ticker']}_predictions.csv")
model_path = os.path.join(config['save_dir'], "stock_model.h5")

# Download stock data
print(f"\n📊 Downloading {config['ticker']} data...")
try:
    stock_data = yf.download(config['ticker'], start=config['start_date'], end=config['end_date'])
    print("✅ Data downloaded successfully!")
except Exception as e:
    print(f"❌ Failed to download data: {e}")
    raise

# Data Processing
stock_data['SMA_10'] = stock_data['Close'].rolling(window=10).mean()
stock_data['SMA_50'] = stock_data['Close'].rolling(window=50).mean()
stock_data.dropna(inplace=True)

# Scale data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(stock_data[config['features']])

# Create sequences
def create_sequences(data, lookback):
    X, y = [], []
    for i in tqdm(range(lookback, len(data)), desc="Creating sequences"):
        X.append(data[i-lookback:i, :])
        y.append(data[i, 0])  # Predict Close price
    return np.array(X), np.array(y)

train_size = int(len(scaled_data) * (1 - config['test_size']))
X_train, y_train = create_sequences(scaled_data[:train_size], config['lookback'])
X_test, y_test = create_sequences(scaled_data[train_size-config['lookback']:], config['lookback'])

# Build and train model
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.3),
    LSTM(64, return_sequences=False),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')

print("\n🚀 Training model...")
history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                   epochs=config['epochs'], batch_size=config['batch_size'], verbose=1)

# Future predictions
def predict_future(model, data, lookback, days_to_predict, scaler, features):
    scaled_data = scaler.transform(data[features].values)
    last_sequence = scaled_data[-lookback:]

    predictions = []
    for _ in tqdm(range(days_to_predict), desc="Predicting future"):
        next_pred = model.predict(last_sequence.reshape(1, *last_sequence.shape))[0, 0]
        predictions.append(next_pred)
        last_sequence = np.roll(last_sequence, -1, axis=0)
        last_sequence[-1, 0] = next_pred

    predictions = scaler.inverse_transform(
        np.concatenate((np.array(predictions).reshape(-1, 1),
                       np.zeros((len(predictions), len(features)-1))), axis=1))[:, 0]

    future_dates = [data.index[-1] + timedelta(days=i) for i in range(1, days_to_predict+1)]
    return future_dates, predictions

future_dates, future_prices = predict_future(
    model, stock_data, config['lookback'],
    config['prediction_days'], scaler, config['features']
)

# Save predictions
predictions_df = pd.DataFrame({
    'Date': future_dates,
    'Predicted_Price': future_prices,
    'Ticker': config['ticker']
})

try:
    predictions_df.to_csv(csv_path, index=False)
    print(f"\n💾 Predictions saved to: {csv_path}")
    files.download(csv_path)
except Exception as e:
    print(f"❌ Failed to save predictions: {e}")
    print("Showing predictions instead:")
    display(predictions_df.head())

# Interactive plot
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=stock_data.index[-100:], y=stock_data['Close'][-100:],
    mode='lines', name='Historical Price'
))
fig.add_trace(go.Scatter(
    x=future_dates, y=future_prices,
    mode='lines+markers', name='Predicted Price'
))
fig.update_layout(
    title=f"{config['ticker']} Price Forecast",
    xaxis_title='Date',
    yaxis_title='Price ($)'
)
fig.show()

/tmp/ipython-input-4052856446.py:50: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


⚠️ Couldn't create directory: [Errno 17] File exists: '/content/sample_data/stock/stock_data.csv'
Using fallback directory: /content

📊 Downloading AAPL data...
✅ Data downloaded successfully!


Creating sequences: 100%|██████████| 272/272 [00:00<00:00, 431323.51it/s]


🚀 Training model...
Epoch 1/10



/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step


Predicting future: 100%|██████████| 30/30 [00:04<00:00,  7.21it/s]


💾 Predictions saved to: /content/AAPL_predictions.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>